In [ ]:
import pyparsing  # make sure you have this installed
import numpy as np
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawHisto import *

from bokeh.io import output_notebook
from Tools.Alice.BetheBloch import *
output_notebook()

# Setup Toy MC

In [ ]:
nPoints=1000000
pdg=ROOT.TDatabasePDG.Instance()
particleList=["e+","mu+","pi+","K+","proton"]
massList=[ pdg.GetParticle(a).Mass() for a in particleList ]    
def GetMass(iPart):
    return [massList[i] for i in iPart]
detectors=["ITS","TPC0","TPC1","TPC2","TRD"]

# Make Toy MC generatinc array of dEdx measurement

In [ ]:
p=np.random.random(nPoints)
p*=5
p+=0.1
particle=np.random.randint(0,5,size=nPoints)
mass=np.asarray(GetMass(particle))
lbg=np.log(p/mass)
data={'p':p, 'particle':particle, 'lbg':lbg}
df=pd.DataFrame(data)
for det in detectors:
    df[det]=BetheBlochAlephNP(lbg)
    df[det]*=np.random.normal(1,0.1,nPoints)

In [ ]:
df.head(10)

# Creating the histogram Array

In [ ]:
histogramArray={}
for det in detectors:
    hisString=det+":p:particle:#"+det+">0>>his"+det+"PP(100,0.5,3,100,0.3,5,5,0,5)"
    ddHis=makeHistogram(df,hisString)
    histogramArray[det]=ddHis
    
 

# Define the parser

In [ ]:
def buildStr(strToBeBuild):
    if isinstance(strToBeBuild,str):
        return strToBeBuild
    iString=''
    iString += '('
    for sub in strToBeBuild:
        iString += buildStr(sub)
    iString += ')'
    return iString

def parseExpression(toParse):
    theContent = pyparsing.Word(pyparsing.alphanums+":,;+/-*^.\/")
    parens = pyparsing.nestedExpr("(", ")", content=theContent)
    res = parens.parseString(toParse)
    projection=res.asList()
    projection[0][0]=buildStr(projection[0][0])
    return projection

# Expression Evaluater

In [ ]:
def evalHistoExpression(expression, histogramArray):
    # expression  hisdY-hisdZ, abs(hisdY-hisdZ)
    print(expression)
    histogram={}
    axes=[]
    varNames=[]
    query=expression[0][0]
    keys=list(set(re.findall(r"\w+",expression[0][0])).intersection(list(histogramArray.keys())))
    func_list = set(re.findall(r"\w+\(",expression[0][0]))  # there still a paranthesis at the end
    
    for iKey in keys:
        query=query.replace(iKey,"histogramArray[\'"+iKey+"\'][\'H\']")
        
        for i,var in enumerate(histogramArray[iKey]["varNames"]):
            try:
                varNames[i].append(var)
            except:
                varNames.append([var])
            varNames[i]=list(set(varNames[i]))
                           
        axes.append(histogramArray[iKey]["axes"])
    
    
    tmp=axes[0]
    for axe in axes:
        for i in range(len(tmp)):
            if not (axe[i] == tmp[i]).all():
                raise valueError("histograms have incompatible axeses." )
    axes=tmp
    
    for iFunc in func_list:
        if iFunc[:-1] in dir(np):
            query=query.replace(iFunc,"np."+iFunc)  
    
    for i,var in enumerate(varNames):
        varNames[i]='-'.join(var)
        
    
    print(query)        
    histogram["H"] = eval(query+"[np.index_exp"+str(expression[0][1]).replace("'","")+"]")
    histogram["name"]=expression[0][0][1:-1]
    print(varNames)
    histogram["varNames"]=varNames
    histogram["axes"]=axes
    
    return histogram

# Draw the histogram

In [ ]:
expression=parseExpression("( (TRD-TRD*0.5+ITS-TRD/2) (0:100,0:10,0:10) (0,1) () )")
#expression=parseExpression("( (ITS) (0:100,0:10,0:10) (0,1) () )")
print(expression)
histo=evalHistoExpression(expression, histogramArray)

In [ ]:
p,d=bokehDrawHistoSliceColz(histo,eval("np.index_exp["+str(expression[0][1][0])+"]"), 0,1,1, {'plot_width':800, 'plot_height':700},{'size':10})
show(p)